<a href="https://colab.research.google.com/github/akhiluthappa1/deeplearning/blob/main/Dimension_reduction_with_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use an arbitrary dataset, except MNIST or another dataset we experimented in the class, e.g. text, image, time series, etc., and do the followings:
###1) Apply SOM, RBM, and one arbitrary Autoencoders (e.g. VAE) to it to reduce the dimension of the original data.

In [ ]:
!pip3 install MiniSom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for MiniSom: filename=MiniSom-2.3.1-py3-none-any.whl size=10609 sha256=083e6096b652728a11583ecb0fad0df7dca4c9b6ff110ddf476347ef571f3a71
  Stored in directory: /root/.cache/pip/wheels/c7/92/d2/33bbda5f86fd8830510b16aa98c8dd420129b5cb24248fd6db
Successfully built MiniSom


In [ ]:
!unzip /content/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: caltech-101/ceiling_fan/image_0030.jpg  
  inflating: caltech-101/ceiling_fan/image_0031.jpg  
  inflating: caltech-101/ceiling_fan/image_0032.jpg  
  inflating: caltech-101/ceiling_fan/image_0033.jpg  
  inflating: caltech-101/ceiling_fan/image_0034.jpg  
  inflating: caltech-101/ceiling_fan/image_0035.jpg  
  inflating: caltech-101/ceiling_fan/image_0036.jpg  
  inflating: caltech-101/ceiling_fan/image_0037.jpg  
  inflating: caltech-101/ceiling_fan/image_0038.jpg  
  inflating: caltech-101/ceiling_fan/image_0039.jpg  
  inflating: caltech-101/ceiling_fan/image_0040.jpg  
  inflating: caltech-101/ceiling_fan/image_0041.jpg  
  inflating: caltech-101/ceiling_fan/image_0042.jpg  
  inflating: caltech-101/ceiling_fan/image_0043.jpg  
  inflating: caltech-101/ceiling_fan/image_0044.jpg  
  inflating: caltech-101/ceiling_fan/image_0045.jpg  
  inflating: caltech-101/ceiling_fan/image_0046.jpg  
  inflating: caltech-101/ceilin

###SOM

In [ ]:
import os
import numpy as np
from PIL import Image
from minisom import MiniSom
import time


# Define the path to the Caltech 101 dataset
path = '/content/caltech-101'

# Function to load the images, resize them and convert them to grayscale
def load_image(path, size):
    img = Image.open(path)
    img = img.resize(size)
    img = img.convert('L') # Convert to grayscale
    img = np.array(img)
    return img

# Function to load the dataset
def load_dataset(path, size):
    dataset = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.jpg'):
                img_path = os.path.join(root, file)
                img = load_image(img_path, size)
                dataset.append(img)
    return np.array(dataset)

start_time = time.time()

# Load the dataset
size = (30, 30) # Size to which the images will be resized
dataset = load_dataset(path, size)

# Flatten the images and normalize the pixel values
dataset = dataset.reshape(dataset.shape[0], -1)
dataset = dataset / 255.

# Define the SOM parameters
map_size = (30, 30) # Size of the SOM grid
input_len = dataset.shape[1] # Length of the input vector
sigma = 1.0 # Standard deviation of the Gaussian function used to initialize the weights
learning_rate = 0.5 # Initial learning rate
num_iterations = 5000 # Number of iterations to train the SOM

# Initialize the SOM
som = MiniSom(map_size[0], map_size[1], input_len, sigma=sigma, learning_rate=learning_rate)
som.random_weights_init(dataset)

# Train the SOM
som.train_random(dataset, num_iterations)

# Get the codebook matrix
codebook = som.get_weights()

end_time = time.time()

# Print the shape of the codebook matrix
print("Original shape of dataset:", dataset.shape)
print("Codebook shape:", codebook.shape)

# total time taken
print("Total time taken:", end_time - start_time)

Original shape of dataset: (9144, 900)
Codebook shape: (30, 30, 900)
Total time taken: 48.51661801338196


In [ ]:
reduced_dataset = np.zeros((dataset.shape[0], size[0]*size[1]))
for i, x in enumerate(dataset):
    # Find the index of the best matching unit in the SOM
    j, k = som.winner(x)
    # Get the weights of the best matching unit, resize it to the original image size and flatten it
    weights = codebook[j][k].reshape(size).flatten()
    # Assign the weights to the corresponding row of the reduced dataset
    reduced_dataset[i] = weights
type(reduced_dataset)

numpy.ndarray

### RBM

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np

# Define a dataset class for the preprocessed images
class ImageDataset(Dataset):
    def __init__(self, path, size):
        self.path = path
        self.size = size
        self.dataset = []
        self.labels = []
        label_idx = 0
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith('.jpg'):
                    img_path = os.path.join(root, file)
                    img = self.load_image(img_path)
                    self.dataset.append(img)
                    self.labels.append(label_idx)
            label_idx += 1
        self.dataset = np.array(self.dataset)
        self.labels = np.array(self.labels)
    def load_image(self, path):
        img = Image.open(path)
        img = img.resize(self.size)
        img = img.convert('L') # Convert to grayscale
        img = np.array(img)
        return img
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        return self.dataset[idx], self.labels[idx]

# Define an RBM model
class RBM(nn.Module):
    def __init__(self, visible_size, hidden_size):
        super().__init__()
        self.W = nn.Parameter(torch.randn(visible_size, hidden_size) * 0.1)
        self.bv = nn.Parameter(torch.zeros(visible_size))
        self.bh = nn.Parameter(torch.zeros(hidden_size))
    def forward(self, x):
        h = torch.sigmoid(torch.matmul(x, self.W) + self.bh)
        x_recon = torch.sigmoid(torch.matmul(h, self.W.T) + self.bv)
        return x_recon, h

start_time = time.time()

# Load the dataset
path = '/content/caltech-101'
size = (50, 50) # Size to which the images will be resized
dataset = ImageDataset(path, size)

# Train the RBM model
rbm = RBM(dataset[0][0].shape[0], 50)
optimizer = optim.Adam(rbm.parameters(), lr=0.001)
criterion = nn.BCELoss()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
for epoch in range(100):
    for batch in dataloader:
        optimizer.zero_grad()
        x, y = batch
        x = x.float() / 255.0
        x_recon, h = rbm(x)
        loss = criterion(x_recon, x)
        loss.backward()
        optimizer.step()

# Encode the images using the RBM weights
X_rbm = torch.sigmoid(torch.matmul(torch.FloatTensor(dataset.dataset), rbm.W) + rbm.bh).detach().numpy()
y_rbm = dataset.labels
print(X_rbm.shape)

end_time = time.time()

# total time taken
print("Total time taken:", end_time - start_time)


(9144, 50, 50)
Total time taken: 166.5788769721985


In [ ]:
y_rbm = y
print(X_rbm.shape, y.shape)

(9144, 50, 50) (9144,)


In [ ]:
type(X_rbm.shape)

tuple

###Autoencoder

In [ ]:
import os
import cv2
import numpy as np
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model

start_time = time.time()

# Set path to the Caltech 101 dataset
dataset_path = "/content/caltech-101"

# Get list of all image file paths
image_paths = []
for dirpath, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        if filename.endswith(".jpg"):
            image_paths.append(os.path.join(dirpath, filename))

# Define dimensions of input and encoded representations
input_dim = (224, 224, 3)
encoding_dim = 64

# Define input layer
input_layer = Input(shape=input_dim)

# Define encoding layers
flatten_layer = Flatten()(input_layer)
hidden_layer = Dense(encoding_dim, activation="relu")(flatten_layer)

# Define decoding layers
output_layer = Dense(np.prod(input_dim), activation="sigmoid")(hidden_layer)
reshape_layer = Reshape(input_dim)(output_layer)

# Define autoencoder model
autoencoder = Model(inputs=input_layer, outputs=reshape_layer)

# Compile autoencoder model
autoencoder.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Define generator to load and preprocess images in batches
def image_generator(image_paths, batch_size):
    i = 0
    while True:
        batch_paths = image_paths[i:i+batch_size]
        batch_images = []
        for image_path in batch_paths:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to RGB format
            img = cv2.resize(img, (224, 224)) # Resize all images to 224x224
            batch_images.append(img)
        i += batch_size
        if i >= len(image_paths):
            i = 0
            np.random.shuffle(image_paths)
        batch_images = np.array(batch_images)
        batch_images = batch_images.astype(np.float32) / 255.0  # Normalize pixel values between 0 and 1
        yield batch_images, batch_images

# Train the autoencoder model using the generator
batch_size = 32
steps_per_epoch = int(np.ceil(len(image_paths) / batch_size))
autoencoder.fit(image_generator(image_paths, batch_size), epochs=5, steps_per_epoch=steps_per_epoch)

# Create encoder model
encoder = Model(inputs=input_layer, outputs=hidden_layer)

# Get the reduced dimensions and labels of all the images in the dataset
reduced_dims = []
labels = []
for image_path in image_paths:
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    reduced_dim = encoder.predict(img)[0]
    reduced_dims.append(reduced_dim)
    label = image_path.split("/")[-2]
    labels.append(label)

# Convert the reduced dimensions and labels to numpy arrays
reduced_dims = np.array(reduced_dims)
labels = np.array(labels)

end_time = time.time()

# total time taken
print("Total time taken:", end_time - start_time)

# Print the shapes of the numpy arrays
print("Shape of the reduced dimensions array:", reduced_dims.shape)
print("Shape of the labels array:", labels.shape)



Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 35ms/step
Total time taken: 887.706383228302
Shape of the reduced dimensions array: (9144, 64)
Shape of the labels array: (9144,)


In [ ]:
reduced_dims

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.0 MB/s eta 0:00:00


### Experiment with three classifier algorithms (XGBoost, LightGBM and CATBoost) and run them on the four datasets. Three dataset includes the original dataset, SOM, RBM, and one Autoencoder. Then compare the results. For classification comparison, you can use accuracy.

### Autoencoder with XGBoost, LightGBM, CATBoost

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from sklearn.preprocessing import LabelEncoder

# Encode class labels using LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(reduced_dims, labels, test_size=0.2, random_state=42)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(reduced_dims, labels, test_size=0.2, random_state=42)

# Train and evaluate XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, y_pred)
print("XGBoost accuracy:", xgb_accuracy)

# Train and evaluate LightGBM model
lgb_model = lgb.LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)
y_pred = lgb_model.predict(X_test)
lgb_accuracy = accuracy_score(y_test, y_pred)
print("LightGBM accuracy:", lgb_accuracy)

# Train and evaluate CATBoost model
cat_model = cat.CatBoostClassifier(random_state=42, verbose=False)
cat_model.fit(X_train, y_train)
y_pred = cat_model.predict(X_test)
cat_accuracy = accuracy_score(y_test, y_pred)
print("CATBoost accuracy:", cat_accuracy)


XGBoost accuracy: 0.0967741935483871
LightGBM accuracy: 0.10716238381629306
CATBoost accuracy: 0.1131765992345544


In [ ]:
!pip install --upgrade xgboost
!pip install --upgrade catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.7 MB/s eta 0:00:00


### Original dataset with XGBoost, LightGBM, and CATBoost

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder



# Load the dataset
dataset_path = "/content/caltech-101"
categories = sorted(os.listdir(dataset_path))

X = []
y = []
for category in categories:
    category_path = os.path.join(dataset_path, category)
    for file_name in os.listdir(category_path):
        image_path = os.path.join(category_path, file_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (64, 64))
        X.append(image)
        y.append(category)

# Convert the data to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
# Preprocess the data using standard techniques
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train and evaluate the XGBoost model
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_accuracy = xgb_model.score(X_test, y_test)

# Train and evaluate the LightGBM model
lgb_model = LGBMClassifier()
lgb_model.fit(X_train, y_train)
lgb_accuracy = lgb_model.score(X_test, y_test)

# Train and evaluate the CATBoost model
cb_model = CatBoostClassifier()
cb_model.fit(X_train, y_train)
cb_accuracy = cb_model.score(X_test, y_test)

# Print the results
print("XGBoost accuracy:", xgb_accuracy)
print("LightGBM accuracy:", lgb_accuracy)
print("CATBoost accuracy:", cb_accuracy)


### SOM with XGBoost, LightGBM, CATBoost

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder


# Calculate the Euclidean distances between each image and each prototype in the codebook matrix
distances = np.zeros((dataset.shape[0], codebook.shape[0]*codebook.shape[1]))
for i, image in enumerate(dataset):
    diff = np.tile(image, (codebook.shape[0]*codebook.shape[1], 1)) - np.repeat(codebook.reshape(-1, dataset.shape[1]), dataset.shape[0], axis=0)
    distances[i] = np.linalg.norm(diff, axis=1)

# Normalize the distances
distances = distances / np.max(distances)

# Split the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(distances, labels, test_size=0.2, random_state=42)

# Train and evaluate the classifiers
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

classifiers = [XGBClassifier(), LGBMClassifier(), CatBoostClassifier()]

for clf in classifiers:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(type(clf).__name__, acc)


ValueError: ignored

RBM

In [ ]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Split the encoded dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_rbm, y_rbm, test_size=0.2, random_state=42)

# Reshape y_train into a 2D matrix with a single column
# y_train = y_train.reshape((-1, 1))
print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
label_map = {label: i for i, label in enumerate(np.unique(y_train))}
y_train = np.array([label_map[label] for label in y_train])
y_test = np.array([label_map[label] for label in y_test])

# Train and evaluate XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': len(np.unique(y_train)),
    'eval_metric': 'merror'
}
num_rounds = 2
xgb_model = xgb.train(params, dtrain, num_rounds)
xgb_preds = xgb_model.predict(dtest)
xgb_acc = accuracy_score(y_test, xgb_preds)
print(f'XGBoost accuracy: {xgb_acc:.4f}')

# Train and evaluate LightGBM
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_test = lgb.Dataset(X_test, label=y_test)
params = {
    'objective': 'multiclass',
    'num_class': len(np.unique(y_train)),
    'metric': 'multi_error',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
}
num_rounds = 2
lgb_model = lgb.train(params, lgb_train, num_rounds, valid_sets=[lgb_test])
lgb_preds = lgb_model.predict(X_test)
lgb_preds = [np.argmax(pred) for pred in lgb_preds]
lgb_acc = accuracy_score(y_test, lgb_preds)
print(f'LightGBM accuracy: {lgb_acc:.4f}')

# Train and evaluate CatBoost
cb_train = cb.Pool(X_train, label=y_train)
cb_test = cb.Pool(X_test, label=y_test)

params = {
    'iterations': 2,
    'learning_rate': 0.1,
    'eval_metric': 'Accuracy',
    'random_seed': 42,
    'loss_function': 'MultiClass',
}

cb_model = cb.CatBoost(params)
cb_model.fit(cb_train, eval_set=cb_test)
cb_preds = cb_model.predict(cb_test)
cb_preds = [np.argmax(pred) for pred in cb_preds]
cb_acc = accuracy_score(y_test, cb_preds)
print(f'CatBoost accuracy: {cb_acc:.4f}')


(7315, 50, 50)
XGBoost accuracy: 0.2149
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.255844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 298623
[LightGBM] [Info] Number of data points in the train set: 7315, number of used features: 2451
[LightGBM] [Info] Start training from score -5.314163
[LightGBM] [Info] Start training from score -5.639586
[LightGBM] [Info] Start training from score -4.635002
[LightGBM] [Info] Start training from score -4.890349
[LightGBM] [Info] Start training from score -4.872331
[LightGBM] [Info] Start training from score -5.762188
[LightGBM] [Info] Start training from score -4.692990
[LightGBM] [Info] Start training from score -5.342334
[LightGBM] [Info] Start training from score -4.553877
[LightGBM] [Info] Start training from score -5.601845
[LightGBM] [Info] Start training from score -4.692990
[LightG